In [1]:
from reframed.io.sbml import load_cbmodel
from cobra.io import read_sbml_model, write_sbml_model
from mewpy.cobra.util import add_enzyme_constraints
from mewpy.simulation import get_simulator
from mewpy.simulation.environment import Environment as Environment
from mewpy.util.request import retreive_gene,retreive_protein
import json

In [13]:
model = load_cbmodel("../models/non-ec/iBth801.xml")

AttributeError: 'NoneType' object has no attribute 'isSetChemicalFormula'

In [14]:
sim = get_simulator(model)
sim.simulate(objective="R_biomass_red")

objective: -0.0
Status: OPTIMAL
Constraints: OrderedDict()
Method:SimulationMethod.FBA

In [15]:
print(sim.genes)

[]


In [ ]:
data_all = dict()
try:
    with open(sim.id+".json", 'r') as f:
        data = json.load(f)
except:
    data = dict()
    for gene in sim.genes:
        try:
            info = retreive_gene(gene,"Escherichia coli")
            # set the inverse kcat to a low value
            data[gene]={'protein':info['protein'],'mw':info['mw']*0.001,'kcat':1e-5}
            data_all[gene]=info
        except:
            data[gene]={'protein': gene[len(sim._g_prefix):], 'mw': 0, 'kcat': 0}
    with open(sim.id+".json", "w") as f:
        json.dump(data,f)

In [ ]:
print("Genes with missing data:")
for k,v in data.items():
    if v['mw']==0:
        print(k)

In [ ]:
print(data)
ec_sim = add_enzyme_constraints(sim, data=data)

In [ ]:
res = ec_sim.simulate()
print(res)